<a href="https://colab.research.google.com/github/thxsxth/POMDP_RLSepsis/blob/master/RL/RL_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

cd 'drive/My Drive/sepsis3-cohort'

/content/drive/My Drive/sepsis3-cohort


In [ ]:
import torch
import numpy as np
import pandas as pd
import datetime as dt
import random
import time
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import torch.nn.functional as F
import os
import glob
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence,pad_packed_sequence
from autoencoder_models import Encoder,patient_encoder,Transition,mechanistic
device='cuda' if torch.cuda.is_available() else 'cpu'
from load_and_process import *

print('Loading Done')

Loading Done


In [ ]:
rl_pats=list(np.load('RL_Pat_list.npy'))
val_pats=pd.read_csv('RL_val.csv',usecols=['pat'])
val_pats=np.unique(val_pats.iloc[:].values.reshape(-1))

In [ ]:
# sofa=pd.read_csv('./pivoted_sofa/sofa_with_vaso3.csv',parse_dates=['endtime','starttime'])
sofa.head()

In [ ]:
sofa=sofa[['starttime','endtime','icustay_id','liver_24hours','cardiovascular_24hours',
                                      'cns_24hours','renal_24hours','SOFA_24hours']]

sofa.head()

,starttime,endtime,icustay_id,liver_24hours,cardiovascular_24hours,cns_24hours,renal_24hours,SOFA_24hours
0,2181-11-25 18:00:00,2181-11-25 19:00:00,200001,0.0,0.0,0.0,2.0,3.0
1,2181-11-25 19:00:00,2181-11-25 20:00:00,200001,0.0,0.0,0.0,2.0,3.0
2,2181-11-25 20:00:00,2181-11-25 21:00:00,200001,0.0,0.0,0.0,2.0,3.0
3,2181-11-25 21:00:00,2181-11-25 22:00:00,200001,0.0,0.0,0.0,2.0,3.0
4,2181-11-25 22:00:00,2181-11-25 23:00:00,200001,0.0,0.0,0.0,3.0,4.0


In [ ]:
labs=labs[['ICUSTAY_ID','charttime','ANIONGAP','BICARBONATE',	'CREATININE',	'CHLORIDE',	'GLUCOSE',	'HEMATOCRIT',	
           'HEMOGLOBIN','PLATELET'	,'POTASSIUM',	'SODIUM',	'BUN'	,'WBC']]


In [ ]:
labs.describe()

In [ ]:
labs=labs[['ICUSTAY_ID','charttime','ANIONGAP','BICARBONATE',	'CREATININE',	'CHLORIDE',	'GLUCOSE',	'HEMATOCRIT',	
           'HEMOGLOBIN','PLATELET'	,'POTASSIUM',	'SODIUM',	'BUN'	,'WBC']]

cols=['ANIONGAP','BICARBONATE',	'CREATININE',	'CHLORIDE',	'GLUCOSE',	'HEMATOCRIT',	
           'HEMOGLOBIN','PLATELET'	,'POTASSIUM',	'SODIUM',	'BUN'	,'WBC']
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
labs[cols]=scaler.fit_transform(labs[cols])

print('Labs Rescaled')

Labs Rescaled


In [ ]:
vaso_cv=pd.read_csv('./pivoted_sofa/Vaso_CV.csv',parse_dates=['charttime'])
vaso_MV=pd.read_csv('./pivoted_sofa/Vaso_MV.csv',parse_dates=['endtime','starttime'])

vaso_cv['starttime']=vaso_cv['charttime']

fluids_cv=pd.read_csv('./pivoted_sofa/fluids_CV.csv',parse_dates=['charttime'])
fluids_MV=pd.read_csv('./pivoted_sofa/fluids_MV.csv',parse_dates=['endtime','starttime'])

fluids_cv['starttime']=fluids_cv['charttime']


In [ ]:

RL_pat_list=list(np.load('RL_Pat_list.npy'))

In [ ]:
non_icu=list(set(patient_list)-set(RL_pat_list))

## Goal is to get a Replay Buffer

Need the states, hourly includes Vitals, Sofa Scores, Labs and the hidden CV hidden Labs

In [ ]:

class Encoder_labs(nn.Module):
  """ Encodes x_{:t} 
      The job of the Encoder is to remember the past observations.
     
  """

  def __init__(self,input_dim,h_dim,n_layers=12, dropout=0.0):
      
      super(Encoder_labs,self).__init__()
      
      self.rnn=nn.GRU(input_dim,h_dim,n_layers,batch_first=True)
      self.dropout=dropout  
      self.n_layers=n_layers
      self.hidden_dim=h_dim
      self.init_weights()
      

  def  init_weights(self):
        for w in self.rnn.parameters(): # initialize the gate weights with orthogonal
            if w.dim()>1:
                weight_init.orthogonal_(w)

  def forward(self,obs,obs_lens,init_h=None, noise=False):

    
    batch_size, max_len, freq=obs.size()  
    obs_lens=torch.LongTensor(obs_lens).to(device)
    obs_lens_sorted, indices = obs_lens.sort(descending=True)
    obs_sorted = obs.index_select(0, indices).to(device)
    
    packed_obs=pack_padded_sequence(obs_sorted,obs_lens_sorted.data.tolist(),batch_first=True)
    # if init_h is None:
    #     init_h=self.init_h

     
    
    hids, h_n = self.rnn(packed_obs) # hids: [B x T x H]  
                                                  # h_n: [num_layers*B*H)
    _, inv_indices = indices.sort()

    hids, lens = pad_packed_sequence(hids, batch_first=True)         
    hids = hids.index_select(0, inv_indices) #B*T*H
    
    
            
    return hids

In [ ]:
class stacked_autoencoder_new(nn.Module):
  """
  Denoising autoencoder adjusted so it will return the hidden state

  """

  def __init__(self,input_dim,h_dims=[512,128,10],n_layers=[12,8,3]):
    
    super(stacked_autoencoder_new,self).__init__()   
    self.encoder1=Encoder_labs(input_dim,h_dims[0],n_layers[0])
    self.encoder2=Encoder_labs(h_dims[0],h_dims[1],n_layers[1])
    self.encoder3=Encoder_labs(h_dims[1],h_dims[2],n_layers[2])
    self.linear=nn.Linear(h_dims[2],input_dim)


  def forward(self,obs,obs_lens):

    out1=self.encoder1(obs,obs_lens)  #B*T*h_1
    out1=pad_sequence(out1,batch_first=True)

    out2=self.encoder2(out1,obs_lens)  #B*T*h_1
    out2=pad_sequence(out2,batch_first=True) #B*T*h_2

    out3=self.encoder3(out2,obs_lens)
    out3=pad_sequence(out3,batch_first=True) #B*T*h_3
    
    output_=self.linear(out3)

    return output_,out3

In [ ]:
stacked_rnn=stacked_autoencoder_new(12,h_dims=[128,64,10],n_layers=[8,5,3])
if device=='cuda':
  stacked_rnn.cuda()

stacked_rnn.load_state_dict(torch.load('./lab_rnn/stacked__lab_rnn0.5.pt',map_location='cpu'))

<All keys matched successfully>

In [ ]:
class state_space_new(nn.Module):
  
  def __init__(self,z_dim,obs_dim,full_dim,h_dim,a_dim,constants_dim):
    
    super(state_space_new,self).__init__()

    self.rnn=Encoder(full_dim,h_dim,a_dim)
    self.trans=Transition(z_dim,a_dim,h_dim,constants_dim)
    self.z_baselines=torch.FloatTensor([70.0,1.0,2.0,0.7,70.0]).to(device)
    self.initial=patient_encoder(constants_dim,z_dim,h_dim)
    
    if device=='cuda':
      self.cuda()

  def forward(self,trajectories,treatments,constants,seq_lens,obs):
      """
      trajectories : Batch of Full_observations B*T*|Full_O|
      seq_lengths :list
      actions : B*T*|A|
      Constants B*|C|
      obs : B*T*|O|
      others : B*T*(|Full_O|-|O|)
      """
      
      T_max =trajectories.size(1)
      batch_size=trajectories.size(0)
      baselines=self.z_baselines.expand(batch_size,-1)
      z_prev,h_prev=self.initial(constants)
    
      h_prev=h_prev.contiguous()
      
      outputs=[]
      cv_params=[]

      a_prev=torch.zeros(treatments.shape[0],treatments.shape[2]).to(device) #B*A
      rnn_output=self.rnn(trajectories,seq_lens,constants,h_prev) 
   
       #rnn_ouput has shape B*T*H

      # we enclose all the sample statements in the model in a plate.
      # this marks that each datapoint is conditionally independent of the others
      
      for t in range(1,T_max+1):

        z_t=self.trans(z_prev,a_prev,rnn_output[:,t-1,:],constants)
        z_scaled=z_t+baselines
        
        o_t=mechanistic(z_scaled) #B*obs_dim
        outputs.append(o_t)
        cv_params.append(z_scaled)
        
        z_prev = z_t
        a_prev=treatments[:,t-1,:]
         
           
      temp=torch.stack(outputs,dim=1).to(device)
      cv=torch.stack(cv_params,dim=1).to(device)
      return temp,cv


model=state_space_new(z_dim=5,obs_dim=4,full_dim=12,h_dim=64,a_dim=2,constants_dim=3)
model.load_state_dict(torch.load('./physio/0.1_denoise_auto_17.763059199264546.pt'))
# model.load_state_dict(torch.load('./Auto Encoder dicts/deterministic_final.pt'))

<All keys matched successfully>

In [ ]:
ls Auto\ Encoder\ dicts

In [ ]:
C1,C2=-0.125,-0.025
def get_rewards(df,dead):
  """
  Get's rewards for a trajectory
  df_sofa: Pandas Df which contains SOFA scores
  MUST BE REINDEXED to have time
  dead (bool): If the terminal is survival or death
  """
  # Calculate rewards for SOFA_{t+1} -SOFA_{t}

  # rewards1=C1*(df.SOFA_24hours-df.shift().SOFA_24hours).dropna().values
  rewards1=C1*(df.SOFA_24hours.values[1:]-df.SOFA_24hours.values[:-1])
  # print(rewards1.shape)
  
  rewards2=C2*((df.shift().SOFA_24hours.iloc[1:]==df.SOFA_24hours.iloc[1:])&(df.SOFA_24hours.iloc[1:]>0)).astype('int').values
  # print(rewards2.shape)
  
  rewards=rewards1+rewards2
  # Calculate Terminal rewards
  if dead:
    rewards=np.concatenate([rewards,[-15]])
  else:
    rewards=np.concatenate([rewards,[15]])

  return rewards


4+4

8

We have a patient, so we need to get the 'hidden states', ( all timed in order) and rewards and buffer should contain (s,a,s',r,done)

For each patient we can get hidden lab states +hidden cv_states

In [ ]:
pat=RL_pat_list[299]

In [ ]:
def get_lab_hs(pat_labs):
   """

   Takes patient specific lab dataframe and returns labs with hidden values concat'd
   Input : pd.Dataframe, index: Charttime, and cols [['charttime','ANIONGAP','BICARBONATE',	'CREATININE',	'CHLORIDE',	'GLUCOSE',	'HEMATOCRIT',	
           'HEMOGLOBIN','PLATELET'	,'POTASSIUM',	'SODIUM',	'BUN'	,'WBC']]
   Output : pd.Dataframe with index:Charttime

   """
   lab_vals=torch.Tensor(pat_labs.values).to(device).unsqueeze(0)
   seq_len=[lab_vals.shape[1]]

   with torch.no_grad():
     _,h=stacked_rnn(lab_vals,seq_len)

   labs_h=pd.DataFrame(np.concatenate([pat_labs.values,h.squeeze(0).to('cpu').numpy()],axis=1),index=pat_labs.index) 

   return labs_h
 
# pat_lab=labs[labs.ICUSTAY_ID==pat]
# pat_lab=pat_lab[['charttime','ANIONGAP','BICARBONATE',	'CREATININE',	'CHLORIDE',	'GLUCOSE',	'HEMATOCRIT',	
#            'HEMOGLOBIN','PLATELET'	,'POTASSIUM',	'SODIUM',	'BUN'	,'WBC']]

# pat_lab=pat_lab.set_index('charttime').resample('12H').mean().ffill().dropna()


# pat_labs=get_lab_hs(pat_lab)
# pat_labs.head()

In [ ]:
def get_trajectory(pat):

  # pat_fluids=input_fluids[input_fluids.icustay_id==pat].set_index('charttime')
  pat_sofa=sofa[sofa.icustay_id==pat].set_index('endtime').resample('H').last().ffill()

  pat_vaso=pd.concat([vaso_cv[vaso_cv.icustay_id==pat].set_index('starttime').resample('H').last(),
           vaso_MV[vaso_MV.icustay_id==pat].set_index('starttime').resample('H').mean().fillna(0)]).resample('H').last().fillna(0)

  pat_vaso['Vaso_rate']=pat_vaso['rate_std']+pat_vaso['vaso_rate']

  pat_fluids=pd.concat([fluids_cv[fluids_cv.icustay_id==pat].set_index('starttime').resample('H').sum(),
           fluids_MV[fluids_MV.icustay_id==pat].set_index('starttime').resample('H').sum().fillna(0)]).resample('H').last().fillna(0)

  pat_fluids['volume']=pat_fluids['tev_mv']+pat_fluids['tev_cv']

 
  
  pat_sofa=pd.concat([pat_sofa,pat_fluids,pat_vaso]).resample('H').sum()[['liver_24hours','cardiovascular_24hours',
                                      'cns_24hours','renal_24hours','SOFA_24hours','Vaso_rate','volume']]

  pat_vitals=vitals[vitals.icustay_id==pat].set_index('charttime')[['HeartRate','SysBP','DiasBP',	'MeanBP','RespRate','SpO2','TempC']]
  

  pat_lab=labs[labs.ICUSTAY_ID==pat]
  pat_lab=pat_lab[['charttime','ANIONGAP','BICARBONATE',	'CREATININE',	'CHLORIDE',	'GLUCOSE',	'HEMATOCRIT',	
           'HEMOGLOBIN','PLATELET'	,'POTASSIUM',	'SODIUM',	'BUN'	,'WBC']]

  pat_lab=pat_lab.set_index('charttime').resample('12H').mean().ffill().dropna()

  if pat_lab.shape[0]<1:
    return None
  
  
  
  labs_with_h=get_lab_hs(pat_lab)



  pat_df=pd.concat([pat_vitals,
                              pat_sofa,labs_with_h]).resample('H').last().ffill().dropna()

 
  dead=co.loc[pat].HOSPITAL_EXPIRE_FLAG==1
  pat_df=pat_df.truncate(before=co.loc[pat].intime-pd.Timedelta('1hr'))

  if co.loc[pat].HOSPITAL_EXPIRE_FLAG==1:
       pat_df=pat_df.truncate(after=co.loc[pat].death_time)
  else:
    pat_df=pat_df.truncate(after=co.loc[pat].outtime)
  if pat_df.shape[0]<2:
    return None

  treatments=torch.FloatTensor(pat_df[['Vaso_rate','volume']].values).to(device).unsqueeze(0)     

  trajectory=torch.FloatTensor(pat_df[['HeartRate','SysBP','DiasBP',	'MeanBP','RespRate','SpO2','TempC',
                                      'liver_24hours','cardiovascular_24hours',
                                      'cns_24hours','renal_24hours','SOFA_24hours']].values).to(device).unsqueeze(0)


  obs=torch.FloatTensor(pat_df[['HeartRate','SysBP','DiasBP',	'MeanBP']].values).to(device).unsqueeze(0)
  constants=torch.FloatTensor(co.loc[pat][['age','is_male','Weight']]).to(device).unsqueeze(0)
  
  seq_len=[pat_df.shape[0]]

  with torch.no_grad():
    _,cvs=model(trajectory,treatments,constants,seq_len,obs)

  full_states=pd.DataFrame(np.concatenate([pat_df.values,cvs[:,:,1:].squeeze(0).to('cpu').numpy()],axis=1),
                         index=pat_df.index,columns=list(pat_df.columns)+['R','C','T','SV'])

  full_states['age']=co.loc[pat].age
  full_states['gender']=co.loc[pat].is_male
  full_states['weight']=co.loc[pat].Weight

  treatments=full_states[['Vaso_rate','volume']].values
  rewards=get_rewards(pat_df,dead)
  full_states=full_states.drop(['Vaso_rate','volume'],axis=1)
  # print(full_states)
  return full_states,treatments,rewards


print('Done')

Done


In [ ]:
buffer=[]
max_sofa=0
pat=0
"""
buffer should contain (s,s',a,r,done)
is done=T, s'=None
"""

for i,pat in enumerate(val_pat_list):
   output=get_trajectory(pat)
   if output:
     full_states,treatments,rewards=output
     T=full_states.shape[0]
    #  this_sofa=max(full_states.SOFA_24hours)
    #  if this_sofa>max_sofa:
    #       max_sofa=this_sofa
    #       print('Max_sofa is',max_sofa,'Patient is :',pat)
    #       pat=pat
     full_states=full_states.values
     if T<2:
       continue
     
     
     for t in range(1,T+1):
        
        a=treatments[t-1,:]
        s=full_states[t-1,:]
        
        r=rewards[t-1]
        if t==T:
            # s1=None
            done=True

        else:
          #  s1=full_states[t,:]
           done=False
        
        # Appends a 5,tuple
        buffer.append((pat,s,a,r,done))

    
   
   print(i/len(val_pat_list)*100,'% Done')


In [ ]:
states=np.vstack([exp[1] for exp in buffer])
states.shape

(2596604, 41)

In [ ]:
actions=np.vstack(([exp[2] for exp in buffer]))
rewards=np.vstack(([exp[3] for exp in buffer]))
rewards.shape

(2596604, 1)

In [ ]:
df=pd.DataFrame(np.hstack([states,actions,rewards]))

In [ ]:
df['pat']=[exp[0] for exp in buffer]
cols="DBP	HR	MBP	RR	SOFA	SpO2	SBP	TempC	cardio	cns	liver	renal	AG	BICARB	CREAT	CL	GLUC	HCT	HB	PLAT	K	Na	BUN	WBC	l_1	l_2	l_3	l_4	l_5	l_6	l_7	l_8	l_9	l_10	R	C	T	SV	age	M/F	weight	Vaso	Fluids	Rewards	Pat".split()
df.columns=cols

In [ ]:
df.to_csv('RL_val.csv')